In [1]:
import os
import subprocess

In [2]:
def run_command(command, max_retries=5,verbose=False):
    """
    Run a shell command locally or over SSH, capturing output and handling errors.
    Retries on 'Connection closed by remote host' errors.
    """
    retries = max_retries if any(cmd in command for cmd in ["ssh", "scp", "ssh-keyscan"]) else 1
    attempts = 0

    while attempts < retries:
        output, success = "", False
        try:
            result = subprocess.run(command, check=True, capture_output=True, text=True)
            output, success = result.stdout.strip(), True
            if verbose:
                print(f"✅ Command executed successfully: {command}")
            return output, success
        except subprocess.CalledProcessError as e:
            error_msg = e.stderr.strip()
            if verbose:
                print(f"❌ Error executing command: {error_msg}")
            
            if "Connection closed by remote host" in error_msg and attempts < retries - 1:
                attempts += 1
                if(verbose):
                    print(f"🔄 Retrying in 5 seconds... (Attempt {attempts}/{retries})")
                time.sleep(5)
            else:
                return error_msg, False  # Return error message and success=False

    return "", False  # Should never reach this

In [3]:
#setup config in .ssh

In [4]:
username="cpignedo"

In [5]:
config=f"""Host login.g100.cineca.it
  Hostname login.g100.cineca.it
  User {username}
  StrictHostKeyChecking no"""

In [6]:
ssh_dir = os.path.expanduser("~/.ssh")
os.makedirs(ssh_dir, exist_ok=True)

config_path = os.path.join(ssh_dir, "config")

with open(config_path, "w") as f:
    f.write(config)

# Ensure correct permissions
#os.chmod(ssh_dir, 0o700)
#os.chmod(config_path, 0o600)

print("Written ~/.ssh/config:")
print("--------------------------------------------------")
print(open(config_path).read())

Written ~/.ssh/config:
--------------------------------------------------
Host login.g100.cineca.it
  Hostname login.g100.cineca.it
  User cpignedo
  StrictHostKeyChecking no


In [7]:
#setup and config g100

In [8]:

setup_command = [
    "verdi", "computer", "setup",
    "--label", 'g100',
    "--hostname", 'login.g100.cineca.it',
    "--description", 'g100 at CINECA',
    "--transport", 'core.ssh_async',
    "--scheduler", 'core.slurm',
    "--shebang", '#!/bin/bash',
    "--work-dir", '/g100_scratch/userexternal/{username}/aiida/',
    "--mpirun-command", 'mpirun -np {tot_num_mpiprocs}',
    "--mpiprocs-per-machine", '128',
#    "--default-memory-per-machine", '68719476736',
    "--prepend-text", '#SBATCH --account=IscrC_PHOTODOP',
    "--non-interactive",
    "--not-use-double-quotes"]

output, success = run_command(setup_command)
if not success:
    print(f"❌ Error setting up computer : {output}")
else:
    print(f"✅ Successfully set up computer g100.")

✅ Successfully set up computer g100.


In [9]:
configure_command = [
    "verdi", "computer", "configure", "core.ssh_async", 'g100',
    "--max-io-allowed", '8',
    "--safe-interval", '30',
    "--use-login-shell",
    "--non-interactive",
]

output, success = run_command(configure_command)
if not success:
    print(f"❌ Error configuring SSH for computer : {output}")
else:
    print(f"✅ Successfully configured SSH for computer g100.") 

✅ Successfully configured SSH for computer g100.


In [10]:
#setup siesta@g100

In [11]:
prepend = """module purge
module load profile/chem-phys
module load autoload
module load siesta/4.1.5"""
code_command = [
    "verdi", "code", "create", "core.code.installed",
    "--computer", 'g100', 
    "--filepath-executable", '/cineca/prod/opt/applications/siesta/4.1.5/intelmpi--oneapi-2021--binary/bin/siesta-master',
    "--label", 'siesta', #relabeled,
    "--description", 'siesta 4.1.5 at CINECA',
    "--default-calc-job-plugin", 'siesta',
    "--with-mpi",
    "--prepend-text", prepend,
    "--append-text",  " ",
    "--no-use-double-quotes"]

output, success = run_command(code_command)
if not success:
    print(f"❌ Error setting up code siesta: {output}")
else:
    print(f"✅ Successfully set up code siesta.")

✅ Successfully set up code siesta.
